<a href="https://colab.research.google.com/github/semthedev/ml-course-2025/blob/main/seminars%5C14_matrix_decompositions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 14. Matrix Decompositions

**Разложение матрицы** — представление матрицы
$A$ в виде произведения матриц, обладающих некоторыми определёнными свойствами (например, ортогональностью, симметричностью, диагональностью).

## General concept of matrix factorization

Матричное разложение используют для следующих задач:
- Решение СЛАУ $Ax = f$
- Вычисления собственных чисел и собственных векторов
- Вычисления сингулярных чисел и сингулярных векторов
- Вычисления обратной матрицы, детерминанта матрицы
- Вычисления **матричных функций**, таких как $\exp(A), \cos(A)$ (это не поэлементные функции)

Для решения этих задач мы представляем исходную матрицу как суму или произведение матриц с более простой "структурой", таким образом, мы сможем решать упомянутые задачи быстрее / в более стабильной форме.

In [4]:
!pip install jupyter_black

In [5]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


# LU разложение

Ограничения: матрица $A$ — квадратная и невырожденная, причём все её ведущие главные миноры отличны от нуля.

$A=LU$,

где $L$ — нижняя треугольная матрица, а $U$ — верхняя треугольная.

**$LU$-разложение** применяется для решения СЛАУ, обращения матриц и вычисления определителя.
$LU$-разложение существует только в том случае, когда матрица $A$ обратима, а все ведущие (угловые) главные миноры матрицы $A$ невырождены.

In [6]:
import numpy as np

a = np.array([[1.0, 1.0], [0.5, 1]])
a0 = a.copy()

n = a.shape[0]
n

2

In [7]:
L = np.zeros_like(a)
U = np.zeros_like(a)

for k in range(n):  # eliminating row k
    L[k, k] = 1.0
    for i in range(k + 1, n):  #
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k + 1, n):
            a[i, j] += -L[i, k] * a[k, j]
    for i in range(k, n):
        U[k, i] = a[k, i]

In [8]:
L

array([[1. , 0. ],
       [0.5, 1. ]])

In [9]:
U

array([[1. , 1. ],
       [0. , 0.5]])

In [10]:
L @ U

array([[1. , 1. ],
       [0.5, 1. ]])

In [11]:
import scipy

scipy.linalg.lu(a0)

(array([[1., 0.],
        [0., 1.]]),
 array([[1. , 0. ],
        [0.5, 1. ]]),
 array([[1. , 1. ],
        [0. , 0.5]]))

In [12]:
rng = np.random.default_rng()
S = scipy.sparse.random(1000, 1000, density=0.75, rng=rng)
A_s = S.toarray()

In [13]:
A_s[1] = A_s[0] * 2000
A_s[4] = A_s[0] * 8000
A_s[5] = A_s[0] * 10
A_s

array([[8.35001654e-01, 0.00000000e+00, 1.79044864e-01, ...,
        1.35713531e-02, 2.36113407e-01, 5.67753487e-01],
       [1.67000331e+03, 0.00000000e+00, 3.58089728e+02, ...,
        2.71427063e+01, 4.72226814e+02, 1.13550697e+03],
       [7.61377234e-01, 0.00000000e+00, 1.17722671e-01, ...,
        1.48079141e-01, 0.00000000e+00, 2.24214357e-01],
       ...,
       [3.35198374e-01, 1.88809209e-01, 4.92071278e-01, ...,
        3.41425752e-01, 5.05083023e-01, 3.67994037e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.55424447e-01, 9.75308768e-01, 2.78496227e-01],
       [1.82377975e-01, 2.86765580e-02, 3.71475258e-01, ...,
        1.81727644e-01, 6.85514130e-01, 0.00000000e+00]])

In [14]:
p, l, u = scipy.linalg.lu(A_s)

In [15]:
np.allclose(A_s, p @ l @ u)

True

In [16]:
abs((A_s - p @ l @ u)).sum()

np.float64(4.0001890626742574e-10)

In [17]:
eps = 1.8e-17
a = np.array([[eps, 1], [1, 1.0]])
L = np.zeros_like(a)
U = np.zeros_like(a)

for k in range(n):  # eliminating row k
    L[k, k] = 1.0
    for i in range(k + 1, n):  #
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k + 1, n):
            a[i, j] += -L[i, k] * a[k, j]
    for i in range(k, n):
        U[k, i] = a[k, i]

In [18]:
L

array([[1.00000000e+00, 0.00000000e+00],
       [5.55555556e+16, 1.00000000e+00]])

In [19]:
U

array([[ 1.80000000e-17,  1.00000000e+00],
       [ 0.00000000e+00, -5.55555556e+16]])

In [20]:
print(L @ U)

[[1.8e-17 1.0e+00]
 [1.0e+00 0.0e+00]]


a# Cholesky (разложение Холецкого, метод квадратного корня)

Представление симметричной положительно определённой матрицы $A$ в виде $A = LL^T$, где $L$ — нижняя треугольная матрица со строго положительными элементами на диагонали.

Эквивалентная форма: $A = U^TU$, где $U = L^T$ — верхняя треугольная матрица.

**Разложение Холецкого** всегда существует и единственно для любой симметричной положительно определённой матрицы.

## Алгоритм

Элементы матрицы $L$ вычисляются начиная с верхнего левого угла матрицы, по формулам:

$l_{11} = \sqrt{a_{11}}$

$l_{j1} = \frac{a_{j1}}{l_{11}}, j \in [2, n]$

$l_{ii} = \sqrt{a_{ii} - \sum_{p = 1}^{i - 1} l_{ip}^2},  i \in [2, n]$

$l_{ji} = \frac{1}{l_{ii}} \left (a_{ji} - \sum_{p = 1}^{i - 1} l_{ip} l_{jp} \right ), i \in [2, n - 1], j \in [i + 1, n]$

Выражение под корнем всегда положительно, если $A$ — действительная положительно определённая матрица.

Вычисление происходит сверху вниз, слева направо, т. е. сперва $L_{ij}$, а затем $L_{ii}$.

Применяется при решении СЛАУ $Ax=b$ (если матрица $A$ симметрична и положительно определена), в методах Монте-Карло для генерации коррелированных случайных величин и т. д.



In [21]:
a = np.array([[eps, 1], [1, 1.0]])
L = np.zeros_like(a)
a, L

(array([[1.8e-17, 1.0e+00],
        [1.0e+00, 1.0e+00]]),
 array([[0., 0.],
        [0., 0.]]))

In [22]:
### THIS DOES NOT WORK
A = a.copy()

for i in range(n):
    for k in range(i + 1):
        s = 0.0
        for j in range(k):
            s += L[i, j] * L[k, j]
        if i == k:
            L[i, i] = np.sqrt(a[i, i] - s)
        else:
            L[i, k] = 1.0 / L[k, k] * (a[i, k] - s)

/tmp/ipython-input-2459015274.py:10: RuntimeWarning: invalid value encountered in sqrt
  L[i, i] = np.sqrt(a[i, i] - s)


In [23]:
L

array([[4.24264069e-09, 0.00000000e+00],
       [2.35702260e+08,            nan]])

In [24]:
L @ L.T

array([[1.8e-17,     nan],
       [    nan,     nan]])

# QR

Представление матрицы в виде произведения **унитарной** (или **ортогональной матрицы**) и **верхнетреугольной матрицы**.

Матрица $A$ размера $n \times m$, где $n \geqslant m$, с комплексными элементами может быть представлена в виде:

$A=QR$

где $Q$ — матрица размера $n \times m$ с ортонормированными столбцами, а $R$ — верхнетреугольная матрица размера $m \times m$.

В случае $n=m$ матрица $Q$ унитарная. Если при этом $A$ невырождена, то $QR$-разложение единственно и матрица $R$ может быть выбрана так, чтобы её диагональные элементы были положительными вещественными числами. В частном случае, когда матрица $A$ состоит из вещественных чисел, матрицы $Q$ и $R$  также могут быть выбраны вещественными, причём $Q$ является ортогональной.

Примечание:
Унитарная матрица - квадратная матрица с комплексными элементами, результат умножения которой на эрмитово сопряжённую равен единичной матрице:
$U†U=UU†=I$

Интерпретация: преобразование, представляемое унитарной матрицей, сохраняет скалярное произведение (а потому и длины всех векторов).



In [25]:
n = 10
r = 6
a = [[1.0 / (i + j + 0.5) for i in range(r)] for j in range(n)]  # Hilbert matrix
a = np.array(a)
print(a)

[[2.         0.66666667 0.4        0.28571429 0.22222222 0.18181818]
 [0.66666667 0.4        0.28571429 0.22222222 0.18181818 0.15384615]
 [0.4        0.28571429 0.22222222 0.18181818 0.15384615 0.13333333]
 [0.28571429 0.22222222 0.18181818 0.15384615 0.13333333 0.11764706]
 [0.22222222 0.18181818 0.15384615 0.13333333 0.11764706 0.10526316]
 [0.18181818 0.15384615 0.13333333 0.11764706 0.10526316 0.0952381 ]
 [0.15384615 0.13333333 0.11764706 0.10526316 0.0952381  0.08695652]
 [0.13333333 0.11764706 0.10526316 0.0952381  0.08695652 0.08      ]
 [0.11764706 0.10526316 0.0952381  0.08695652 0.08       0.07407407]
 [0.10526316 0.0952381  0.08695652 0.08       0.07407407 0.06896552]]


In [26]:
q, Rmat = np.linalg.qr(a)
e = np.eye(r)
print("Built-in QR orth", np.linalg.norm(q.T @ q - e))

Built-in QR orth 8.933073361428516e-16


In [27]:
gram_matrix = a.T.dot(a)
Rmat1 = np.linalg.cholesky(gram_matrix)
q1 = np.linalg.solve(Rmat1, a.T).T
print("Via Cholesky:", np.linalg.norm(q1.T @ q1 - e))

Via Cholesky: 1.655777363169412e-05


In [28]:
a = np.array([[1.0, 0], [0, 1.0]])
scipy.linalg.cosm(a)

array([[0.54030231, 0.        ],
       [0.        , 0.54030231]])

# LAPACK

* [LinearRegression](https://github.com/scikit-learn/scikit-learn/blob/6e9039160/sklearn/linear_model/_base.py#L465) fit method:
```python
        if self.positive:
            self.coef_ = optimize.nnls(X, y)[0]
            ...
        elif sp.issparse(X):
            ...
            self.coef_ = lsqr(X_centered, y)[0]
            ...
        else:
            self.coef_, _, self.rank_, self.singular_ = linalg.lstsq(X, y)
            self.coef_ = self.coef_.T
```
NNLS was your in bonus hw, so lets discover what `lsqr` and `lstsq` are.

* [scipy.linalg.lstsq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html#lstsq)
```python
    lapack_func, lapack_lwork = get_lapack_funcs((driver,
                                                 '%s_lwork' % driver),
                                                 (a1, b1))
    if driver in ('gelss', 'gelsd'):
        ...                    
        if driver == 'gelss':
            lwork = _compute_lwork(lapack_lwork, m, n, nrhs, cond)
            v, x, s, rank, work, info = lapack_func(a1, b1, cond, lwork,
                                                    overwrite_a=overwrite_a,
                                                    overwrite_b=overwrite_b)
    
        elif driver == 'gelsd':
            ...
            lwork, iwork = _compute_lwork(lapack_lwork, m, n, nrhs, cond)
             x, s, rank, info = lapack_func(a1, b1, lwork,
                                            iwork, cond, False, False)
            
    return x, resids, rank, s
    ...
    elif driver == 'gelsy':
        lwork = _compute_lwork(lapack_lwork, m, n, nrhs, cond)
        jptv = np.zeros((a1.shape[1], 1), dtype=np.int32)
        v, x, j, rank, info = lapack_func(a1, b1, jptv, cond,
                                          lwork, False, False)
        return x, np.array([], x.dtype), rank, None
```

[LAPACK](https://en.wikipedia.org/wiki/LAPACK) (Linear Algebra PACKage) — библиотека с открытым исходным кодом, содержащая методы для решения основных задач линейной алгебры и написанная на FORTRAN. В LAPACK реализованы алгоритмы для решения систем линейных уравнений, задач на собственные значения и разложения по сингулярным значениям, реализации матричных разложений, таких как LU, QR, разложение Холецкого и т. д.

Процедуры в LAPACK имеют стандартизированные названия:

A LAPACK subroutine name is in the form `pmmaaa`, where:

- `p` is a one-letter code denoting the type of numerical constants used. `S`, `D` `C` and `Z`.
- `mm` is a two-letter code denoting the kind of matrix expected by the algorithm. When the code `GE` (General) is given, the subroutine expects an $n×n$ array containing the entries of the matrix.
- `aaa` is a one- to three-letter code describing the actual algorithm implemented in the subroutine.


Сайт LAPACK: https://netlib.sandia.gov/lapack/#_presentation

### `gelss`
Computes the minimum norm solution to a real linear least squares problem using the singular value decomposition (SVD).

### `gelsd`
The problem is solved in three steps:

- (1) Reduce the coefficient matrix A to bidiagonal form with
     Householder transformations, reducing the original problem
     into a "bidiagonal least squares problem" (BLS)
- (2) Solve the BLS using a divide and conquer approach.
- (3) Apply back all the Householder transformations to solve
     the original least squares problem.

### `gelsy`
The routine first computes a QR factorization with column pivoting:
```
     A * P = Q * [ R11 R12 ]
                 [  0  R22 ]
```
 with R11 defined as the largest leading submatrix whose estimated
 condition number is less than 1/RCOND.  The order of R11, RANK,
 is the effective rank of A.

 Then, R22 is considered to be negligible, and R12 is annihilated
 by orthogonal transformations from the right, arriving at the
 complete orthogonal factorization:
 ```
    A * P = Q * [ T11 0 ] * Z
                [  0  0 ]
```
 The minimum-norm solution is then
```
    X = P * Z**T [ inv(T11)*Q1**T*B ]
                 [        0         ]
```
 where Q1 consists of the first RANK columns of Q.

# Tasks

### Bonus task 13: gelss (easy)
Implement `gelss` (with numpy, python and scipy.svd).


In [29]:
import numpy as np
from scipy.linalg import svd, lstsq


def gelss(A, b, rcond=None):
    U, S, Vt = svd(A, full_matrices=False)
    m, n = A.shape

    if rcond is None:
        eps = np.finfo(S.dtype).eps
        rcond = max(m, n) * eps

    tol = rcond * S.max()
    large = S > tol
    rank = int(np.count_nonzero(large))
    S_inv = np.zeros_like(S)
    S_inv[large] = 1.0 / S[large]

    if b.ndim == 1:
        c = U.T @ b
        w = S_inv * c
        x = Vt.T @ w
    else:
        c = U.T @ b
        w = (S_inv[:, None]) * c
        x = Vt.T @ w

    return x, S, rank

In [ ]:
# пример почекать
A = np.array([[1,2],[2,1],[3,1]], float)
b = np.array([1,2,2], float)

# my gelss
x_gelss, S_gelss, rank_gelss = gelss(A, b)

# scipy lstsq
x_scipy, res_scipy, rank_scipy, s_scipy = lstsq(A, b, lapack_driver='gelss')

print("gelss solution:", x_gelss)
print("gelss singular values:", S_gelss)
print("gelss rank:", rank_gelss , "\n")

print("scipy lstsq solution:", x_scipy)
print("scipy residuals:", res_scipy)
print("scipy rank:", rank_scipy)
print("scipy singular values:", s_scipy)

gelss solution: [0.68571429 0.2       ]
gelss singular values: [4.2499715  1.39202811]
gelss rank: 2 

scipy lstsq solution: [0.68571429 0.2       ]
scipy residuals: 0.2571428571428572
scipy rank: 2
scipy singular values: [4.2499715  1.39202811]


### Bonus task 14: gelsd
Implement `gelsd` (with numpy and python ans scipy.svd).


In [30]:
def gelsd(A, b, rcond=None):
    A = np.asarray(A)
    b = np.asarray(b)

    m, n = A.shape

    U, s, Vt = svd(A, full_matrices=False)

    if rcond is None:
        eps = np.finfo(s.dtype).eps
        rcond = max(m, n) * eps
    tol = rcond * s.max() if s.size else 0.0

    large = s > tol
    rank = int(np.count_nonzero(large))

    s_inv = np.zeros_like(s)
    s_inv[large] = 1.0 / s[large]

    if b.ndim == 1:
        c = U.T @ b
        w = s_inv * c
        x = Vt.T @ w
    else:
        c = U.T @ b
        w = (s_inv[:, None]) * c
        x = Vt.T @ w

    if b.ndim == 1:
        r = b - A @ x
        if m > n and rank == n:
            residuals = np.array([np.dot(r, r)])
        else:
            residuals = np.array([], dtype=A.dtype)
    else:
        R = b - A @ x
        if m > n and rank == n:
            residuals = np.sum(R * R, axis=0)
        else:
            residuals = np.array([], dtype=A.dtype)

    return x, residuals, rank, s

In [31]:
A = np.array([[1, 2], [2, 1], [3, 1]], float)
b = np.array([1, 2, 2], float)

x, residuals, rank, s = gelsd(A, b)

x_scipy, res_scipy, rank_scipy, s_scipy = lstsq(A, b, lapack_driver="gelsd")

print("gelss solution:", x)
print("residuals:", residuals)
print("gelss singular values:", s)
print("gelss rank:", rank, "\n")

print("scipy lstsq solution:", x_scipy)
print("scipy residuals:", res_scipy)
print("scipy rank:", rank_scipy)
print("scipy singular values:", s_scipy)

gelss solution: [0.68571429 0.2       ]
residuals: [0.25714286]
gelss singular values: [4.2499715  1.39202811]
gelss rank: 2 

scipy lstsq solution: [0.68571429 0.2       ]
scipy residuals: 0.2571428571428572
scipy rank: 2
scipy singular values: [4.2499715  1.39202811]


### Bonus task 15: gelsy
Implement `gelsy` (with numpy, python and scipy.qr).

In [38]:
from scipy.linalg import qr


def gelsy(A, b, rcond=None):
    A = np.asarray(A)
    b = np.asarray(b)
    m, n = A.shape

    b_was_1d = b.ndim == 1
    if b_was_1d:
        B = b[:, None]
    else:
        B = b
    if rcond is None:
        eps = np.finfo(A.dtype if np.issubdtype(A.dtype, np.floating) else float).eps
        rcond = max(m, n) * eps

    # Rank-revealing QR with column pivoting
    Q, R, piv = qr(A, mode="economic", pivoting=True)
    k = min(m, n)

    # Estimating rank from diagonal of R
    diagR = np.abs(np.diag(R)) if k > 0 else np.array([])
    if diagR.size == 0:
        rank = 0
    else:
        tol = rcond * diagR.max()
        rank = int(np.sum(diagR > tol))

    if rank == 0:
        X = np.zeros((n, B.shape[1]), dtype=A.dtype)
        return (X[:, 0] if b_was_1d else X, np.array([], dtype=A.dtype), 0, None)
    Q1 = Q[:, :rank]
    R1 = R[:rank, :]
    Qz, Rz = qr(R1.T, mode="full", pivoting=False)
    T = Rz[:rank, :]
    T11 = T.T
    Y = Q1.T @ B
    U1 = np.linalg.solve(T11, Y)
    U = np.vstack([U1, np.zeros((n - rank, B.shape[1]), dtype=A.dtype)])
    X_piv = Qz @ U
    X = np.zeros_like(X_piv)
    X[piv, :] = X_piv
    residuals = np.array([], dtype=A.dtype)

    return (X[:, 0] if b_was_1d else X, residuals, rank, s)

In [39]:
A = np.array([[1, 2], [2, 1], [3, 1]], float)
b = np.array([1, 2, 2], float)

x, residuals, rank, s = gelsy(A, b)

x_scipy, res_scipy, rank_scipy, s_scipy = lstsq(A, b, lapack_driver="gelsy")

print("gelss solution:", x)
print("residuals:", residuals)
print("gelss singular values:", s)
print("gelss rank:", rank, "\n")

print("scipy lstsq solution:", x_scipy)
print("scipy residuals:", res_scipy)
print("scipy rank:", rank_scipy)
print("scipy singular values:", s_scipy)

gelss solution: [0.68571429 0.2       ]
residuals: []
gelss singular values: None
gelss rank: 2 

scipy lstsq solution: [0.68571429 0.2       ]
scipy residuals: []
scipy rank: 2
scipy singular values: None


### Bonus task 16: Cholesky
Fix bugs in the Cholesky decomposition algorithm.

In [71]:
import numpy as np


def cholesky(a):
    n = a.shape[0]
    L = np.zeros_like(a)
    D = np.zeros(n)

    for i in range(n):
        for k in range(i + 1):
            s = 0.0
            for j in range(k):
                s += L[i, j] * D[j] * L[k, j]
            if i == k:
                D[i] = a[i, i] - s
                L[i, i] = 1.0
            else:
                L[i, k] = (a[i, k] - s) / D[k]

    print("L @ L.T = ", L @ np.diag(D) @ L.T)

    return L

In [72]:
a = np.array([[eps, 1], [1, 1.0]])
cholesky(a)

L @ L.T =  [[1.8e-17 1.0e+00]
 [1.0e+00 0.0e+00]]


array([[1.00000000e+00, 0.00000000e+00],
       [5.55555556e+16, 1.00000000e+00]])

In [73]:
L

array([[4.24264069e-09, 0.00000000e+00],
       [2.35702260e+08,            nan]])

**TODO:** [scipy.sparse.linalg.lsqr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lsqr.html)


---

This notebook was adapted from **Mathematics for Machine Learning** book by Marc Deisenroth, Aldo Faisal and Cheng Ong and Ivan Oseledec **course on numerical linear algebra**, and the **LAPACK documentation**.